<a href="https://colab.research.google.com/github/JeonJunWooo/mini_project/blob/main/%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B83%EC%B0%A8_AI_%EC%98%81%EC%83%81%EC%BD%94%EB%84%88%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 안녕하세요^^ 
# AIVLE 미니 프로젝트3차 팀 미션입니다. 
# 주어진 문제를 잘 읽고 해결하시길 바랍니다.


## 올레TV 예능 콘텐츠 중 하나인 프로그램 "코미디빅리그"의 영상을 분석하고 코너를 분류하는 AI 모델을 만들어 보세요.

"코미디빅리그" 는 tvN에서 방영 중인 서바이벌 형식의 공개 코미디 프로그램으로 3개월 단위로 인기 코너를 선정하는 방식인 쿼제를 도입하고 있다. 
이번 과제는 2022년 1쿼터(1~3월) 방영분 중 2월 방영한 3편을 학습 데이터로 하여 3월 방송의 코너 구간을 추출해내는 것입니다.

아래의 조건들을 참고하여 문제를 해결하세요.
> - 주어진 데이터 ( Train 데이터 :  2월 방영분 3편 / Test 데이터 : 3월 방영분 1편 )
> - 제출 파일은 <font color="red">.ipynb 파일</font>과  조에서 각 과정마다 문제 해결을 위해 진행한 방법을 정리한 <font color="red">발표자료(.pptx)</font>를 포함히여 제출합니다.<br> 
①데이터 분석 및 학습데이터 생성, ②데이터 전처리, ③AI 모델링(아키텍처, 성능비교), ④분류 결과 <br> 
※ 각 과정은 문제 해결 방법에 따라 변경 가능<br><br>
> <b>제출 파일( 압축파일명 : [팀]미니프로젝트3차_AI 0반_00조.zip) </b> <br>
>> ① 코드 : [팀]미니프로젝트3차_AI 0반_00조.ipynb <br>
>> ② 발표자료 : [팀]미니프로젝트3차_AI 0반 00조.pptx <br>




---

# 0. 환경 설정하기

### 1) 구글 드라이브 연결하기

In [1]:
# 코랩 사용 시 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 2) 경로 확인하기

In [2]:
# ROOT_PATH 확인 
import os

if os.getcwd() == '/content' :
  # 구글 드라이브 사용 시 
  ROOT_PATH = "/content/drive/MyDrive/AIVLE_3rd_Team"
else :
  ROOT_PATH = os.path.abspath('..')

# 비디오 파일이 저장되어 있는 폴더
VIDEO_PATH = ROOT_PATH + "/video"

# 모델 학습을 위한 데이터 
TRAIN_PATH = ROOT_PATH + "/train"
# 모델 예측을 위한 test 데이터
TEST_PATH = ROOT_PATH + "/test"

# 테스트 영상 test.mp4 경로 
TEST_VIDEO = '/content/drive/MyDrive/AIVLE_3rd_Team/video/test' + "/220320.mp4"

# 모델 예측을 위해 테스트 영상을 프레임 이미지로 자르고 저장하는 경로 (testGenerator 생성 시 PATH )   
TEST_IMAGE = TEST_PATH + "/image"

# 모델(.h5) 파일이 저장된 경로 (본인이 생성한 모델)
MODEL_PATH = ROOT_PATH +  '/model/팀_미니프로젝트3차_a206205_전준우.h5'

### 3) 영상 데이터 다운로드하기
아래 코드를 실행하세요.

In [ ]:
# # test data 다운로드
# !pip install gdown

# import gdown
# import shutil

# google_path = 'https://drive.google.com/uc?id='
# file_id = '....................'
# output_name = 'video.zip'

# gdown.download(google_path+file_id,output_name,quiet=False)

# shutil.move("./" + output_name, ROOT_PATH)

# import zipfile

# with zipfile.ZipFile(ROOT_PATH + "/video.zip") as z:
#     z.extractall(ROOT_PATH)


# 미니프로젝트 3차 팀 미션 지금부터 시작하세요!

In [3]:
# 여기에서부터 입력하세요.

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.backend import clear_session
from tensorflow.keras.models import Sequential, Model

from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.losses import categorical_crossentropy, sparse_categorical_crossentropy

from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
from keras.layers import BatchNormalization, Dropout

import random as rd
from sklearn.metrics import accuracy_score
from tensorflow.keras import layers


In [4]:
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator

In [5]:
TEST_IMAGE

'/content/drive/MyDrive/AIVLE_3rd_Team/test/image'

In [6]:
TEST_VIDEO

'/content/drive/MyDrive/AIVLE_3rd_Team/video/test/220320.mp4'

## test영상 1초단위로 잘라내서 저장

In [35]:
%cd /content/drive/MyDrive/AIVLE_3rd_Team/test
!unzip -qq "/content/drive/MyDrive/AIVLE_3rd_Team/test/image.zip"

/content/drive/MyDrive/AIVLE_3rd_Team/test


In [13]:
IMAGE_PATH = TEST_IMAGE

TIME_MEASUERMENT_UNIT = 1 #TIME MEASUREMENT UNIT을 통해 몇 초 단위로 이미지를 저장할 지 선택

if not os.path.exists(IMAGE_PATH):
    os.mkdir(IMAGE_PATH)

video = cv2.VideoCapture(TEST_VIDEO)

while video.isOpened():
    ret,frame = video.read()
    if ret:
        # 현재 프레임 위치 (msec) 
        frame_sec = video.get(cv2.CAP_PROP_POS_MSEC)/1000
        if frame_sec.is_integer():
            if (frame_sec % TIME_MEASUERMENT_UNIT == 0):
                filename = IMAGE_PATH + "/" + str(round(frame_sec)) + ".jpg"
                cv2.imwrite(filename, frame) 
    else:
        break

video.release()

## 동일영역 잘라내고 test 해보기

In [20]:
!pip install natsort

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:
from natsort import natsort

In [36]:
def croppedImage(TEST_IMAGE):
  #이미지를 잘라낼 영역 
  bounding_box = {
      'start_x' : 40, 
      'start_y' : 60,
      'end_x' : 157,    
      'end_y' : 75
  }
  from tqdm.auto import tqdm, trange
  image_list = glob.glob('/content/drive/MyDrive/AIVLE_3rd_Team/test/image' + '/*.jpg')
  image_list = natsort.natsorted(image_list)
  for image in tqdm(image_list, desc="IMAGE CROP PROGRESS"):
    original_image = cv2.imread(image)
    cropped_image = original_image[bounding_box['start_y']:bounding_box['end_y'], bounding_box['start_x']:bounding_box['end_x']]    
    cv2.imwrite(image, cropped_image) 

In [39]:
croppedImage(TEST_IMAGE)

IMAGE CROP PROGRESS:   0%|          | 0/3182 [00:00<?, ?it/s]

## 확인해보기

In [ ]:
image_list = glob.glob('/content/drive/MyDrive/AIVLE_3rd_Team/test/image' + '/*.jpg')
image_list = natsort.natsorted(image_list)

In [40]:
def my_preprocessing(TEST_VIDEO):

    batch_size = 16
    img_height = 480
    img_width = 854


    test_datagen = ImageDataGenerator(
    rescale=1. / 255,  # 이미지 데이터 정규화
    )

    test_generator = test_datagen.flow_from_directory(
    TEST_PATH,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale',
    shuffle=False
    )


    return test_generator

In [41]:
test_generator = my_preprocessing(TEST_VIDEO)

Found 3182 images belonging to 2 classes.


In [42]:
import h5py

In [54]:
def my_model_predict(test_generator, MODEL_PATH):

    class_map = {
    0: 'chorus',
    1: 'enter',
    2: 'merry',
    3: 'two',
    }

    model = tf.keras.models.load_model(MODEL_PATH)
    pred = model.predict(test_generator)
    predidxs = np.argmax(pred, axis=1)

    prediction = []
    for i in predidxs:
        prediction.append(class_map[i])

    import pandas as pd
    predict_df = pd.DataFrame({'sec': range(1,3183),
                               'predict': prediction})
    predict_df = predict_df.set_index('sec')


    return predict_df

In [55]:
predict_df = my_model_predict(test_generator, MODEL_PATH)

In [56]:
predict_df

,predict
sec,
1,enter
2,enter
3,enter
4,enter
5,enter
...,...
3178,enter
3179,enter
3180,enter


In [76]:
opening = [ 'opneing' for i in range(1,129)]
enter = ['enter' for i in range(129,1039)]
chorus = ['chorus' for i in range(1039,1556)]
merry = ['merry' for i in range(1556,2358)]
two = ['two' for i in range(2358,2950)]
ending = ['ending' for i in range(2950,3183)]
corner = opening + enter + chorus + merry + two + ending

result = {
    'sec' : range(1,3183)
    , 'corner':corner
}

df_result = pd.DataFrame(result)
df_result

pj3 = pd.read_csv('/content/drive/MyDrive/AIVLE_3rd_Team/predict_df.csv')

pj3 = pd.merge(pj3, df_result, on='sec')
pj3['same'] = np.where(pj3['predict'] == pj3['corner'], 0, 1)

pj3.loc[129:2948]

pj3.loc[129:2948].groupby(by='same')['sec'].count()
pj3.loc[129:1038].groupby(by='same', as_index=False)['sec'].count()

tmp_enter = pd.DataFrame(pj3.loc[129:1038].groupby(by='same', as_index=False)['sec'].count())
tmp_enter.rename(columns={'sec':'enter'}, inplace=True)

tmp_chorus = pd.DataFrame(pj3.loc[1038:1554].groupby(by='same', as_index=False)['sec'].count())
tmp_chorus.rename(columns={'sec':'chorus'}, inplace=True)

tmp_merry = pd.DataFrame(pj3.loc[1554:2356].groupby(by='same', as_index=False)['sec'].count())
tmp_merry.rename(columns={'sec':'merry'}, inplace=True)

tmp_two = pd.DataFrame(pj3.loc[2356:2948].groupby(by='same', as_index=False)['sec'].count())
tmp_two.rename(columns={'sec':'two'}, inplace=True)

tmp_1 = pd.merge(tmp_enter, tmp_chorus, on='same', how='outer')
tmp_2 = pd.merge(tmp_1, tmp_merry, on='same', how='outer')
tmp = pd.merge(tmp_2, tmp_two, on='same', how='outer')

tmp['total'] = tmp['enter'] + tmp['chorus'] + tmp['merry'] + tmp['two']
tmp.loc[2] = [2,0,0,0,0,0]
tmp['same'] = tmp['same'].map({0:'same', 1:'diffrence', 2:'accuracy'})
tmp.loc[2, 'enter'] = round(tmp.loc[0, 'enter'] / (tmp.loc[0, 'enter'] + tmp.loc[1, 'enter']) * 100, 2)
tmp.loc[2, 'chorus'] = round(tmp.loc[0, 'chorus'] / (tmp.loc[0, 'chorus'] + tmp.loc[1, 'chorus']) * 100, 2)
tmp.loc[2, 'merry'] = round(tmp.loc[0, 'merry'] / (tmp.loc[0, 'merry'] + tmp.loc[1, 'merry']) * 100, 2)
tmp.loc[2, 'two'] = round(tmp.loc[0, 'two'] / (tmp.loc[0, 'two'] + tmp.loc[1, 'two']) * 100, 2)
tmp.loc[2, 'total'] = round(tmp.loc[0, 'total'] / (tmp.loc[0, 'total'] + tmp.loc[1, 'total']) * 100, 2)


In [77]:
pj3

,sec,predict,corner,same
0,1,enter,opneing,1
1,2,enter,opneing,1
2,3,enter,opneing,1
3,4,enter,opneing,1
4,5,enter,opneing,1
...,...,...,...,...
3177,3178,enter,ending,1
3178,3179,enter,ending,1
3179,3180,enter,ending,1
3180,3181,enter,ending,1


In [78]:
tmp

,same,enter,chorus,merry,two,total
0,same,909.00,513.00,801.00,590.00,2813.00
1,diffrence,1.00,4.00,2.00,3.00,10.00
2,accuracy,99.89,99.23,99.75,99.49,99.65


In [46]:
len(os.listdir('/content/drive/MyDrive/AIVLE_3rd_Team/train/image/chorus'))

144

In [47]:
len(os.listdir('/content/drive/MyDrive/AIVLE_3rd_Team/train/image/enter'))

177

In [48]:
len(os.listdir('/content/drive/MyDrive/AIVLE_3rd_Team/train/image/marry'))

149

In [49]:
len(os.listdir('/content/drive/MyDrive/AIVLE_3rd_Team/train/image/two'))

156

In [58]:
predict_df.to_csv("/content/drive/MyDrive/AIVLE_3rd_Team/predict_df.csv")